In [54]:
import numpy as np
import pandas as pd
import re
from collections import OrderedDict
from collections import Counter
from itertools import combinations
from math import log
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

# DATA

In [22]:
product_descriptions = pd.read_csv('DATA/product_descriptions.csv', header=0, encoding = "ISO-8859-1")
attributes = pd.read_csv('DATA/attributes.csv', header=0, encoding = "ISO-8859-1")
train = pd.read_csv('DATA/train.csv', header=0,  encoding = "ISO-8859-1")
test = pd.read_csv('DATA/test.csv', header=0,  encoding = "ISO-8859-1")

# EVALUATOR SCORES (NORMALIZED)
* Evaluator scores are are continuous averages on the interval [1, 3]; the classifier seems unable to handles this
* relevance_rounded consists of values: 1, 2, 3
* relevance_rounded_quarters consists of values 1, 1.25, 1.5...3

In [23]:
# Since relevance is an aggregate, values are continuous rather than discrete
# It appears the classifier can only handle a limited number of values
# So we round up
train['relevance_rounded'] = train['relevance'].map(lambda x: round(x))

In [24]:
def round_custom(x):
    '''Rounds relevance to quarters -- for greater granularity'''
    whole, fraction = int(x), 100 * float(x - int(x))

    if fraction < 12.5:   fraction =  0
    elif fraction < 37.5: fraction = 25
    elif fraction < 62.5: fraction = 50
    elif fraction < 87.5: fraction = 75
    else: fraction = 100

    return whole + fraction / 100

In [25]:
train['relevance_rounded_quarters'] = train['relevance'].map(lambda x: round_custom(x))

# FEATURES

### FEATURE: EXACT MATCH
* between search term (as phrase) and product title, allowing only differences in case

In [26]:
# Produces a boolean

train['exact'] = pd.Series([True if st.lower() in pt.lower() else False 
                      for st, pt in zip(train['search_term'], train['product_title'])])

### FEATURE: OVERLAPPING WORDS
* allowing for differences in case

In [27]:
# Produces a count >= 0

train['overlapping_words'] = pd.Series([len(set(st.lower().split()) & set(pt.lower().split()))
                      for st, pt in zip(train['search_term'], train['product_title'])])

### FEATURE: PERCENTAGE OVERLAPPING WORDS
* allowing for differences in case

In [28]:
train['percentage_overlapping_words'] = pd.Series([len(set(st.lower().split()) & 
                                                       set(pt.lower().split()))/float(len(st.split()))
                                                       for st, pt in zip(train['search_term'], train['product_title'])])

## TF-IDF

In [29]:
def calc(doc, query, N=1, D={}):
    '''Calculates TF-IDF score for doc, query pair, given a document count (N) and dictionary of word counts (D)'''
    doc = [w.lower() for w in doc.split()]
    counts = Counter(doc)
    query = query.lower().split()
    return sum([counts[word] * log(N / (1+D[word])) for word in set(query) & set(doc)]) # 1 to avoid div by 0

### TF-IDF: product descriptions (all) + product titles

In [30]:
# Number of documents
N = len(set(product_descriptions['product_uid'] + train['product_uid']))

In [31]:
# Dictionary of word-count pairs
words_product_descriptions = ''.join(product_descriptions['product_description']).split()
words_train = ''.join(train['product_title']).split()
D = Counter(words_product_descriptions + words_train) # Not lowercased

In [32]:
# Test
# frequent = sorted(d.items(), key=lambda x: x[1])[:50:-1]
# frequent

In [33]:
scores = OrderedDict()

for row in zip(train['id'], train['product_uid'], train['product_title'], train['search_term']):
    _id, pid, pt, st = row
    TF = Counter(pt.split())
    score = sum([calc(pt, word, N, D) for word in st.lower().split()])
    scores[_id] = score

In [34]:
# Vector
tf_idf = list(scores.values())
train['tf_idf'] = tf_idf
train.ix[0]

id                                                              2
product_uid                                                100001
product_title                   Simpson Strong-Tie 12-Gauge Angle
search_term                                         angle bracket
relevance                                                       3
relevance_rounded                                               3
relevance_rounded_quarters                                      3
exact                                                       False
overlapping_words                                               1
percentage_overlapping_words                                  0.5
tf_idf                                                    4.16932
Name: 0, dtype: object

## TF-IDF: product descriptions (not all -- only those with product titles) + product titles

## TF-IDF: product titles alone (no descriptions)

## TF-IDF: product titles, descriptions, and attributes

## TF-IDF sans stopwords (does it make any difference?)

In [35]:
# Dictionary of word-count pairs
words_product_descriptions = ''.join(product_descriptions['product_description']).split()
words_train = ''.join(train['product_title']).split()
d = Counter(words_product_descriptions + words_train) # Not lowercased

In [36]:
# Function for calculating TF-IDF score for doc, query pair

def calc(doc, query):
    doc = [w.lower() for w in doc.split()]
    counts = Counter(doc)
    query = query.lower().split()
    return sum([counts[word] * log(N / (1+d[word])) for word in set(query) & set(doc)]) # 1 to avoid div by 0

In [37]:
scores = OrderedDict()

for row in zip(train['id'], train['product_uid'], train['product_title'], train['search_term']):
    _id, pid, pt, st = row
    TF = Counter(pt.split())
    score = sum([calc(pt, word) for word in st.lower().split()])
    scores[_id] = score

## TF-IDF sans stopwords (any difference?)

In [38]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

## FEATURE: JACCARD DISTANCE

In [39]:
# def jaccard_dist(phrase1, phrase2):
#     '''Returns the Jaccard distance for two phrases, eg, search query and product title'''
#     lst1, lst2 = set(phrase1.lower().split()), set(phrase2.lower().split())
#     return float(len(lst1 & lst2)) / len(lst1 | lst2)

# def words_shared(st, pt):
#     st_list = re.split('\s', st.lower())
#     pt_list = re.split('\s', pt.lower())
#     dist = jaccard_dist(st_list, pt_list)
#     return dist

# train['jaccard'] = pd.Series([words_shared(st, pt) 
#                       for st, pt in zip(train['search_term'], train['product_title'])])

### BAG OF WORDS: product descriptions, attributes, titles -- product by product

### DEFAULT PENALTY: products that appear too many times -- and have low relevance -- suggesting that they are defaults for when the search engine is stumped

In [40]:
# Aggregate on product ID's themselves and penalize too popular products.
from collections import Counter
d = Counter(train['product_uid'])
for pid in d: 
    if d[pid] > 5:
        pass
#         print(pid, d[pid],) 

# THE MODEL

## Train, test split

In [42]:
# Prepare to divide the dataset into test, train (lifted from sklearn Iris example)
train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75

# Split data into training and test
train_data, test_data = train[train['is_train']==True], train[train['is_train']==False]

## F1 Score

In [49]:
def f1(ct, preds):
    '''Returns the F1 score given the confusion matrix generated with a set of predictions'''
    index = list(pd.crosstab(test_data['relevance_rounded'], preds))
    total = 0
    for i in list(ct):
        total += sum(list(ct[i]))

    true_positives, false_positives, true_negatives, false_negatives = 0, 0, 0, 0

    for i in zip(list(ct), range(1, len(ct)+1)): 
        row = list(ct.ix[i[1]])
        column = list(ct[i[0]])
    
        tp = row[i[1]-1]
        true_positives  += tp
        false_negatives += sum(row)-tp
        false_positives += sum(column) - tp
        true_negatives = total - tp
    
    return float(2*true_positives) / (2 * true_positives + false_positives + false_negatives)

In [53]:
train.columns

Index(['id', 'product_uid', 'product_title', 'search_term', 'relevance',
       'relevance_rounded', 'relevance_rounded_quarters', 'exact',
       'overlapping_words', 'percentage_overlapping_words', 'tf_idf',
       'is_train'],
      dtype='object')

## RANDOM FOREST

In [55]:
def random_forest(features):
    # clf = RandomForestClassifier(n_jobs=2)
    clf = RandomForestClassifier(n_jobs=2, class_weight = 'balanced', oob_score = False, criterion = 'entropy')

    y, _ = pd.factorize(train_data['relevance_rounded']) # 0, 1, 2

    clf.fit(train_data[features], y)

    # Predicting on those features will output predictions that match y
    preds = clf.predict(test_data[features])

    # target_names = test['relevance_rounded']
    target_names = ['1', '2', '3']
    out = [target_names[pred] for pred in preds]

    # preds = target_names[clf.predict(test[features])]

    preds = clf.predict(test_data[features])

    pd.crosstab(test_data['relevance_rounded'], np.asarray(out), rownames=['actual'], colnames=['preds'])
    
    return (ct, preds)

In [60]:
# Features
train.columns[7:-1]

Index(['exact', 'overlapping_words', 'percentage_overlapping_words', 'tf_idf'], dtype='object')

In [67]:
for n in range(1, 5):
    for comb in combinations([7, 8, 9, 10], n):
#         print(comb)
        features = train.columns[list(comb)]
        ct, preds = random_forest(features)
#         print(comb, f1(ct, preds))
        print(ct)


preds      1     2     3
actual                  
1        280   159   873
2       2292  2167  4086
3       1830  4030  2749
preds      1     2     3
actual                  
1        280   159   873
2       2292  2167  4086
3       1830  4030  2749
preds      1     2     3
actual                  
1        280   159   873
2       2292  2167  4086
3       1830  4030  2749
preds      1     2     3
actual                  
1        280   159   873
2       2292  2167  4086
3       1830  4030  2749
preds      1     2     3
actual                  
1        280   159   873
2       2292  2167  4086
3       1830  4030  2749
preds      1     2     3
actual                  
1        280   159   873
2       2292  2167  4086
3       1830  4030  2749
preds      1     2     3
actual                  
1        280   159   873
2       2292  2167  4086
3       1830  4030  2749
preds      1     2     3
actual                  
1        280   159   873
2       2292  2167  4086
3       1830  4030  2749


# THINGS TO TRY / CONTEMPLATE

In [19]:
# What do 3's have in common?
# What do 1's have in common?
# QUERIES CONTAINING WORDS NOT IN DICTIONARY